# 01 - Ingestion Kafka

Producer qui récupère les données de l'API OpenSky Network et les envoie à Kafka.

## Configuration

In [1]:
%pip install -r requirements.txt

import threading
import time
import json
import requests
from kafka import KafkaProducer
from datetime import datetime
import os
from dotenv import load_dotenv

load_dotenv()

KAFKA_BOOTSTRAP = os.getenv("KAFKA_BOOTSTRAP", "kafka1:9092")
TOPIC_NAME = os.getenv("TOPIC_NAME", "opensky-data")

print(f"✅ Kafka: {KAFKA_BOOTSTRAP}")
print(f"✅ Topic: {TOPIC_NAME}")

Ignoring kafka-python: markers 'python_version < "3.12"' don't match your environment
  Cloning https://github.com/dpkp/kafka-python.git to /tmp/pip-install-dwdviqa2/kafka-python_5aec89396abf46c4b1b3639e7b56897d
  Running command git clone --filter=blob:none --quiet https://github.com/dpkp/kafka-python.git /tmp/pip-install-dwdviqa2/kafka-python_5aec89396abf46c4b1b3639e7b56897d
  Resolved https://github.com/dpkp/kafka-python.git to commit f8a91ec4d57caa5440d496c5dab937b7c4498e9a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.
✅ Kafka: kafka1:9092
✅ Topic: opensky-data


## Producer

In [ ]:
stop_producer = False

def run_producer():
    """Récupère les données OpenSky et les envoie à Kafka."""
    print(f"🚀 Producer démarré...")
    
    try:
        producer = KafkaProducer(
            bootstrap_servers=KAFKA_BOOTSTRAP,
            value_serializer=lambda v: json.dumps(v).encode('utf-8')
        )
    except Exception as e:
        print(f"❌ Erreur connexion Kafka: {e}")
        return

    api_url = "https://opensky-network.org/api/states/all"

    while not stop_producer:
        try:
            response = requests.get(api_url)
            if response.status_code == 200:
                data = response.json()
                states = data.get('states') or []
                timestamp = data['time']

                for s in states:
                    record = {
                        "time": timestamp, "icao24": s[0],
                        "callsign": s[1].strip() if s[1] else None,
                        "origin_country": s[2], "time_position": s[3],
                        "last_contact": s[4], "longitude": s[5], "latitude": s[6],
                        "baro_altitude": s[7], "on_ground": s[8], "velocity": s[9],
                        "true_track": s[10], "vertical_rate": s[11],
                        "geo_altitude": s[13], "squawk": s[14], "spi": s[15],
                        "position_source": s[16],
                        "category": s[17] if len(s) > 17 else None
                    }
                    producer.send(TOPIC_NAME, record)
                
                producer.flush()
                print(f"📡 {len(states)} vols envoyés ({datetime.now().strftime('%H:%M:%S')})")
            else:
                print(f"⚠️ API Status: {response.status_code}")
        except Exception as e:
            print(f"⚠️ Erreur: {e}")
        time.sleep(15)
    
    print("🛑 Producer arrêté.")

producer_thread = threading.Thread(target=run_producer, daemon=True)
producer_thread.start()
print("✅ Producer en arrière-plan")

🚀 Producer démarré...
✅ Producer en arrière-plan


📡 9929 vols envoyés (15:37:41)
📡 9918 vols envoyés (15:37:58)
📡 9910 vols envoyés (15:38:14)
📡 9913 vols envoyés (15:38:32)
📡 9914 vols envoyés (15:38:51)
📡 9916 vols envoyés (15:39:08)
📡 9919 vols envoyés (15:39:26)
📡 9925 vols envoyés (15:39:45)
📡 9922 vols envoyés (15:40:05)
📡 9922 vols envoyés (15:40:23)
📡 9930 vols envoyés (15:40:42)
📡 9921 vols envoyés (15:41:01)
📡 9915 vols envoyés (15:41:20)
📡 9901 vols envoyés (15:41:39)
📡 9912 vols envoyés (15:41:56)
📡 9922 vols envoyés (15:42:14)
📡 9946 vols envoyés (15:42:32)
📡 9945 vols envoyés (15:42:50)
📡 9947 vols envoyés (15:43:12)
📡 9955 vols envoyés (15:43:31)
📡 9960 vols envoyés (15:43:50)
📡 9959 vols envoyés (15:44:08)
📡 9949 vols envoyés (15:44:27)
📡 9928 vols envoyés (15:44:50)
📡 9915 vols envoyés (15:45:12)
📡 9908 vols envoyés (15:45:33)
📡 9929 vols envoyés (15:45:53)
📡 9939 vols envoyés (15:46:17)
📡 9904 vols envoyés (15:46:42)
📡 9894 vols envoyés (15:47:05)
📡 9899 vols envoyés (15:47:25)
📡 9906 vols envoyés (15:47:47)
📡 9920 v

## Arrêt du Producer

In [5]:
stop_producer = True
print("🛑 Arrêt demandé")

🛑 Arrêt demandé
